### What is Modal Platform?
Modal is a cloud platform that simplifies deploying and scaling machine learning inferencing workloads. 

Modal lets you focus on your models while it handles infrastructure, scaling, and cost optimization automatically.
### Setting Up Modal
Follow these steps to get started with Modal:
#### 1. Create an Account
- Visit [modal.com](https://modal.com) and sign up for a new account.
#### 2. Install the Modal Python Package
```bash
pip install modal
```
#### 3. Authenticate with Modal
```bash
modal setup
```
or
```bash
python -m modal setup
```

In [ ]:
import modal

app = modal.App("My-App")

@app.function()
def sum(x: int, y: int) -> None:
    print( x + y )

@app.function()
def square(x: int) -> None:
    print( x ** 2 )


@app.local_entrypoint()
def main(x: int) -> None:
    square.local(x)

## Entrypoints for Ephemeral Apps
The code that runs first when you do ```modal run``` is called the "entrypoint"
### Argument parsing
```bash
    modal run script.py --x 4
```
To run a specific function locally
```bash
    modal run script.py::sum --x 5 --y 8
```
### Deploy a function
```bash
    modal deploy script.py
```
Then in terminal or another python App
```bash
    python
    >>> import modal
    >>> sum_function = modal.Function.from_name("My-App", "sum")
    >>> sum_function.remote(6, 7)
```

## Define Infrastructure then Running it locally and in the cloud 

In [5]:
import modal
from modal import App, Image

app = App("Location-Function")

@app.function(image = Image.debian_slim().pip_install("requests"))
def my_location():
    import requests

    location_data = requests.get('http://ip-api.com/json').json()

    city, country, ip_address = location_data['city'], location_data['country'], location_data['query']
    temperature = requests.get(f"https://wttr.in/{city}?format=%t&m").text.strip()

    response = f"Code running on IP {ip_address} ({city}, {country}) Outside is: {temperature}"
    print(response)

    return response


with modal.enable_output():
    with app.run():
        my_location.local()
        # my_location.remote()

### API endpoints
To turn this function into a web endpoint run: 
```bash 
    modal serve web_api_function.py
```
In the output, you should see a URL that ends with ```hello-dev.modal.run```

If you add ```/docs``` to the end of the URL you can also find interactive documentation, powered by OpenAPI and Swagger

By running the endpoint with ```modal serve```, you created a temporary endpoint that will disappear if you interrupt your terminal

To deploy this endpoint permanently, run 
```bash
    modal deploy web_api_function.py
```

In [ ]:
import modal
from modal import App, Image

app = App(image = Image.debian_slim().pip_install("fastapi[standard]"))

@app.function()
@modal.web_endpoint(docs=True)
def greet(user: str) -> str:
    return f"Hello {user}!"

@app.function()
@modal.web_endpoint(method="POST", docs=True)
def square(item: dict):
    return {"value": item['x']**2}

### Deploy in a Class to Handle expensive startup (heavy model loading)

```@modal.enter()``` lifecycle hook happens after container started

In [ ]:
import modal
from modal import App, Image

app = App("web-app", image = Image.debian_slim().pip_install("fastapi[standard]") )

@app.cls(cpu=1, memory="1Gi")
class WebApp:
    @modal.enter()
    def startup(self):
        from datetime import datetime, timezone

        print("Container started -> Start up time initiated!")
        self.start_time = datetime.now(timezone.utc)

    @modal.method()
    def ping(self):
        return "pong"

    @modal.web_endpoint(docs=True)
    def web(self):
        from datetime import datetime, timezone

        current_time = datetime.now(timezone.utc)
        return {"start_time": self.start_time, "current_time": current_time}

### Keeping it alive
Ater some time, if API is not used, Modal will kill the containers along with our application

We need to create a periodinc cron job that will be pinging it to refresh it's state

In [ ]:
import modal

WebApp = modal.Cls.from_name("web-app", "WebApp")
web_app = WebApp()

reply = web_app.ping.remote()
print(reply)

### Deploying a HuggingFace model in a Modal Class
Possible Auth Error:
```
Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted and you are not in the authorized list. Visit https://huggingface.co/google/gemma-2-2b-it to ask for access.
```
Follow suggested link and click on ```Accept terms / Grant access``` or similar promts that you will see on a model page

Run
```bash
    modal deploy model_class.py
```

In [ ]:
import modal
from modal import App, Image

app = App("gemma-webapp")
image = (
    Image.debian_slim()
    .apt_install("git")
    .pip_install("torch", "transformers", "huggingface_hub", "fastapi[standard]", "accelerate")
    .run_commands("git config --global credential.helper store")
)
secrets = [modal.Secret.from_name("hf-token")]

@app.cls(image=image, secrets=secrets, gpu="T4", container_idle_timeout=1200)
class GemmaModelApp:
    @modal.enter()
    def startup(self):
        import os, torch
        from transformers import AutoTokenizer, AutoModelForCausalLM
        from huggingface_hub import login

        hf_token = os.environ['HF_TOKEN']
        login(hf_token, add_to_git_credential=True)

        print("Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
        print("Loading model...")
        self.model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", device_map="auto")

        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right" 

        print("Model loaded successfully!")

    @modal.method()
    def ping(self):
        from datetime import datetime, timezone
        return f"pong@{datetime.now(timezone.utc)}"
        
    @modal.method()
    def generate(self, prompt: str) -> str:
        print(f"Received prompt: {prompt}")
        return self._generate_response(prompt)

    @modal.web_endpoint(method="POST", docs=True)
    def web_generate(self, prompt: str) -> str:
        print(f"Web Controller Received prompt: {prompt}")
        return self._generate_response(prompt)
    
    def _generate_response(self, prompt: str) -> str:
        import torch
        
        inputs = self.tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        attention_mask = torch.ones(inputs.shape, device="cuda")
        outputs = self.model.generate(
            inputs,
            attention_mask=attention_mask,
            max_new_tokens=500,     # Generate up to 100 tokens
            num_return_sequences=1, # Ensures that the model generates only one response. Increase this number if you want the model to generate multiple variations of the response.
            do_sample=True,         # Enables sampling of (temperature, top_k, top_p) parameters for more diverse responses
            temperature=0.7,        # Controls randomness (lower is more deterministic)
            top_k=50,               # Only considers the top 50 tokens with the highest probabilities
            top_p=0.9,              # Implements nucleus sampling for more diverse and natural responses.
        )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=False)

and for the state refreshing cron job:
```bash
    python cron_job.py
```

In [ ]:
import time
import modal
from datetime import datetime, timezone

GemmaModelApp = modal.Cls.from_name("gemma-webapp", "GemmaModelApp")
gemma_service = GemmaModelApp()

while True:
    reply = gemma_service.ping.remote()
    print(f"ping@{datetime.now(timezone.utc)}: {reply}")
    time.sleep(600)

# Volumes
Modal Volumes allow you to efficiently save and access large files, such as machine learning model weights, across different applications and sessions. 

This capability ensures that your trained models are readily available for fast deployment and inference without the need for repeated uploads or processing.

### Saving models weight to a Volume, then reading it from there and initializing weight for the inference

In [ ]:
from pathlib import Path
from modal import App, Volume, Image

app = App("Volumes-test")

MODEL_DIR = Path("/models")

image = (
    Image.debian_slim()
    .pip_install("huggingface_hub[hf_transfer]","transformers","torch", "accelerate")  
    .env({"HF_HUB_ENABLE_HF_TRANSFER": "1"}) 
)
volume = Volume.from_name("testing-model-weights-directory", create_if_missing=True)

@app.function(
    volumes={MODEL_DIR: volume},  # "mount" the Volume, sharing it with your function
    image=image,  
)
def download_model(
    repo_id: str="facebook/opt-125m", # small model for testing purposes
    revision: str=None,  # include specific revision (commit hash)
    ):
    from huggingface_hub import snapshot_download

    model_path = MODEL_DIR / repo_id

    if not model_path.exists() or not any(model_path.glob("*")):  # directory exists and contains some content (files)
        print(f"Model not found at {model_path}, downloading...")

        snapshot_download(repo_id=repo_id, local_dir=model_path, revision=revision)

        print(f"Model downloaded to {model_path}")
    else:
        print(f"Model already exists at {model_path}")

    from transformers import AutoTokenizer, AutoModelForCausalLM
    import torch

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    prompt = "Hello, how are you?"

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    inputs = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones(inputs.shape)

    output = model.generate(inputs, attention_mask=attention_mask, max_new_tokens=55, pad_token_id=tokenizer.eos_token_id)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Generated Text:", generated_text)

## Construct a vLLM Engine and Deploy It

vLLM is an advanced, high-performance serving framework designed for large language models (LLMs). It optimizes model execution by leveraging a cutting-edge, tensor-parallel engine and dynamic memory management to efficiently handle complex queries. By supporting models such as OpenAI's GPT and LLaMA, vLLM offers faster response times and better scalability, making it ideal for real-time LLM applications. It integrates seamlessly with popular machine-learning frameworks and emphasizes low latency and high throughput for serving language models in production environments.


The vLLM server, which is compatible with OpenAI, is presented as a FastAPI router. 

The process begins by creating an `AsyncLLMEngine`, which is the central component of the vLLM server. 

This engine handles model loading, executes inference operations, and facilitates response delivery.

### Model Download Service
Run
```bash
    modal deploy download-model-service.py
```

In [ ]:
from modal import App, Volume, Image, Secret
from pathlib import Path

MODELS_DIR = "/llm"
DEFAULT_NAME = "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16"
DEFAULT_REVISION = "a7c09948d9a632c2c840722f519672cd94af885d"
MINUTES = 60
HOURS = 60 * MINUTES

volume = Volume.from_name("llm", create_if_missing=True)
image = (
    Image.debian_slim(python_version="3.10")
    .pip_install(["huggingface_hub", "hf-transfer"])
    .env({"HF_HUB_ENABLE_HF_TRANSFER": "1"}) # enable the faster download method
)

app = App("download-model-service",
    image=image,
    secrets=[Secret.from_name("hf-token", required_keys=["HF_TOKEN"])]
)

@app.function(
        volumes={MODELS_DIR: volume}, 
        timeout=4 * HOURS ) # how long the function will run before being terminated
def download_model(model_name: str = DEFAULT_NAME, 
                   model_revision: str = DEFAULT_REVISION, 
                   force_download: bool = False):
    from huggingface_hub import snapshot_download

    volume.reload() # Reload the volume to ensure it is available

    snapshot_download(
        model_name,
        local_dir=MODELS_DIR + "/" + model_name,
        ignore_patterns=[
            "*.pt",
            "*.bin",
            "*.pth",
            "original/*",
        ],
        revision=model_revision,
        force_download=force_download,
    )
    
    completion_file = Path(MODELS_DIR) / model_name / "download_complete"
    completion_file.touch()  # Create an empty file to signal completion

    volume.commit() # Commit changes to volume to ensure visibility to other functions
    

@app.local_entrypoint()
def main(model_name: str = DEFAULT_NAME, model_revision: str = DEFAULT_REVISION, force_download: bool = False):
    download_model.remote(model_name, model_revision, force_download)

### Inferencing Service

In [ ]:
import modal
from pathlib import Path

vllm_image = modal.Image.debian_slim(python_version="3.12").pip_install(
    "vllm==0.6.3post1", "fastapi[standard]==0.115.4"
)

MODELS_DIR = "/llm"
MODEL_REPO_ID = "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16" # quantized to 4-bit
MODEL_REVISION = "a7c09948d9a632c2c840722f519672cd94af885d"

try:
    volume = modal.Volume.lookup("llm", create_if_missing=False)
except modal.exception.NotFoundError:
    raise RuntimeError("Volume not found. Please deploy [download-model-service] first")

app = modal.App("vllm-chat")

N_GPU = 1  # first upgrade to more powerful GPUs, and only then increase GPU count
TOKEN = "my-token"  # auth token, for production use, replace with a modal.Secret

MINUTE = 60
HOUR = 60 * MINUTE


@app.function(
    image=vllm_image,
    gpu=modal.gpu.T4(count=N_GPU),
    container_idle_timeout=5 * MINUTE, # how long the container will wait for new requests before shutting down
    volumes={MODELS_DIR: volume},
    timeout=24 * HOUR, # how long the function will run before being terminated
    allow_concurrent_inputs=1000 # how many requests can be processed concurrently
)
@modal.asgi_app()
def serve():
    import fastapi, time
    from pathlib import Path
    import vllm.entrypoints.openai.api_server as api_server
    from vllm.engine.arg_utils import AsyncEngineArgs
    from vllm.engine.async_llm_engine import AsyncLLMEngine
    from vllm.entrypoints.logger import RequestLogger
    from vllm.entrypoints.openai.serving_chat import OpenAIServingChat
    from vllm.entrypoints.openai.serving_completion import OpenAIServingCompletion
    from vllm.entrypoints.openai.serving_engine import BaseModelPath
    from vllm.usage.usage_lib import UsageContext

    volume.reload() 

    model_path = Path(MODELS_DIR) / MODEL_REPO_ID
    if not model_path.exists() or not any(model_path.glob("*")): 
        print(f"Model not found at {model_path}, downloading...")

        download_model_function = modal.Function.from_name("download-model-service", "download_model")
        download_model_function.remote(MODEL_REPO_ID, MODEL_REVISION)

        while not is_model_downloaded(model_path):
            print(f"Model not downloaded yet, waiting...")
            time.sleep(5)
            volume.reload() 

        print(f"Model downloaded to {model_path}")
    else:
        print(f"Model already exists at {model_path}")


    web_app = fastapi.FastAPI(
        title=f"OpenAI-compatible {MODEL_REPO_ID} server",
        description="Run an OpenAI-compatible LLM server with vLLM on modal.com 🚀",
        version="0.0.1",
        docs_url="/docs",
    )

    http_bearer = fastapi.security.HTTPBearer(
        scheme_name="Bearer Token",
        description="See code for authentication details.",
    )
    web_app.add_middleware(
        fastapi.middleware.cors.CORSMiddleware,
        allow_origins=["*"],
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    async def is_authenticated(api_key: str = fastapi.Security(http_bearer)):
        if api_key.credentials != TOKEN:
            raise fastapi.HTTPException(
                status_code=fastapi.status.HTTP_401_UNAUTHORIZED,
                detail="Invalid authentication credentials",
            )
        return {"username": "authenticated_user"}

    router = fastapi.APIRouter(dependencies=[fastapi.Depends(is_authenticated)])

    router.include_router(api_server.router)
    web_app.include_router(router)

    engine_args = AsyncEngineArgs(
        model=MODELS_DIR + "/" + MODEL_REPO_ID,
        tensor_parallel_size=N_GPU, # allows model computations to be distributed across multiple GPUs for better performance
        gpu_memory_utilization=0.90, #  allowing the engine to use up to 90% of available GPU memory to balance between performance and resource allocation
        max_model_len=8096, # maximum number of tokens the model can process in a single request
        enforce_eager=False, 
    )

    engine = AsyncLLMEngine.from_engine_args(
        engine_args, usage_context=UsageContext.OPENAI_API_SERVER
    )

    model_config = get_model_config(engine)

    request_logger = RequestLogger(max_log_len=2048)

    base_model_paths = [
        BaseModelPath(name=MODEL_REPO_ID.split("/")[1], model_path=MODEL_REPO_ID)
    ]

    api_server.chat = lambda s: OpenAIServingChat(
        engine,
        model_config=model_config,
        base_model_paths=base_model_paths,
        chat_template=None,
        response_role="assistant",
        lora_modules=[],
        prompt_adapters=[],
        request_logger=request_logger,
    )
    api_server.completion = lambda s: OpenAIServingCompletion(
        engine,
        model_config=model_config,
        base_model_paths=base_model_paths,
        lora_modules=[],
        prompt_adapters=[],
        request_logger=request_logger,
    )

    return web_app

# The function call aims to obtain the model's configuration from the engine. 
# This configuration may include parameters like the model's architecture, 
# input/output size, supported features, or other meta-information about the model.
def get_model_config(engine):
    import asyncio

    try:
        event_loop = asyncio.get_running_loop()
    except RuntimeError:
        event_loop = None

    if event_loop is not None and event_loop.is_running():
        model_config = event_loop.run_until_complete(engine.get_model_config())
    else:
        model_config = asyncio.run(engine.get_model_config())

    return model_config

def is_model_downloaded(model_path: Path) -> bool:
    completion_file = model_path / "download_complete"
    return completion_file.exists()